In [ ]:
from src import NeuralTester, ExperimentConfig
import torch
from src.learner import PymooLearner
from src.learner.auxiliary_components import PYMOO_NSGA2_DEFAULT_PARAMS
from src.objective_functions import uqi, confidence_balance
from models import load_stylegan
import wandb
from _enable_logging import enable_logging

## Example usage of the Neural Tester using a MOO
We use wandb to log progress in various steps!

In [ ]:
wandb.login()
enable_logging() # Setup logging to be able to view logging outputs in notebooks.

In [ ]:
# Define the configurations for our experiments.
conf = ExperimentConfig(
    samples_per_class=10,
    generations=50,
    mix_dim_range=(0, 8),
    predictor=torch.load("../models/wrn_mnist.pkl"),  # The System under test (SUT)
    generator=load_stylegan("../models/sg2_mnist.pkl"),  # The generator network (a stylegan in this case)
    metrics=[uqi, confidence_balance],
    classes=10,
)

In [ ]:
learner_params = PYMOO_NSGA2_DEFAULT_PARAMS
learner_params["n_var"] = conf.genome_size
learner = PymooLearner(**learner_params)  # The learner for search based optimization of candidates.

In [ ]:
# Here we initialize the Tester object.
tester = NeuralTester(
    config=conf,
    learner=learner,
    device=torch.device("cuda"),
)

In [ ]:
# We start the testing procedure.
tester.test()